In [ ]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
from matplotlib import pyplot
from dotenv import load_dotenv
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
import numpy as np
import os

Section - Setup Circuit Parameters

In [ ]:
load_dotenv()
workspace = Workspace.from_connection_string(os.environ['azure_connection'])
provider = AzureQuantumProvider(workspace)
# Selecting a backend
# Use simulators to test before running it on real hardware.
backend = provider.get_backend("rigetti.sim.qvm")

qubits = 6 # The number of physical qubits
a = 42 # the hidden integer

# Ensure it can be represented with the number of specified qubits
a = a % 2**(qubits)

Section - Quantum Register and Classical Register Initialization

In [ ]:
qr = QuantumRegister(qubits)
cr = ClassicalRegister(qubits)

circ = QuantumCircuit(qr, cr)

Section - Superposition State Preparation (Equal weights to all basis states)

In [ ]:
for i in range(qubits):
    circ.h(qr[i])  # Apply Hadamard gates to put qubits in supe

circ.barrier()

Section - Oracle Function

In [ ]:
for i in range(qubits):
    # Apply Z gate if the i-th bit of 'a' is 1
    if (a & (1 << i)):
        circ.z(qr[i]) 
    # Apply identity gate if the i-th bit of 'a' is 0
    else:
        circ.id(qr[i])
        circ.barrier()

for i in range(qubits):
    circ.h(qr[i])

Section - Running the circuit

In [ ]:
circ.barrier()
circ.measure(qr,cr)
circ.draw("mpl")


qc_compiled = transpile(circ, backend)
job_sim = backend.run(qc_compiled, shots=1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)

Section - Plotting the results

In [ ]:
plot_histogram(counts)